We use a slight modification of the code in Assignment 3 to define an MDP

In [1]:
type MDP 
    stateSize  :: Int
    actionSize :: Int
    bellmanUpdate! :: Function
    c :: Array{Float64, 2}
    P :: Array{ Array{Float64, 2}, 1}
    
    function MDP(c, P)
        (n, m) = size(c)
        
        if length(P) != m
            error("Number of transition matrices does not match the number of actions")
        end
        
        P_concatenated = vcat(P...)
        if size(P_concatenated) != (n*m, n)
            error("Size of transition and reward matrices are inconsistent")
        end
        
        is_square(Pi) = size(Pi) == (n,n)
        is_row_stochastic(Pi) = isapprox(sum(Pi,2), ones(n); atol=100*eps(Float64))
        is_stopping_action(Pi) = Pi - zero(Pi)
        
        for Pi in P
            if !is_square(Pi)
                error("Transition matrix is not a square matrix")
                print(size(Pi))
                print(n)
            elseif !(is_row_stochastic(Pi) || is_stopping_action(Pi))
                error("Transition matrix is not row stochastic")
            end
        end
        
        function update!(v, g, vOld; discount=1)
            Q = c + discount * reshape(P_concatenated * vOld, n, m)
            
            for x=1:n
                g[x], v[x] = 1, Q[x,1]
                for u=2:m
                    if Q[x,u] < v[x]
                        g[x], v[x] = u, Q[x,u]
                    end
                end
            end
        end
        
        new(n, m, update!, c, P)
    end
end

Now we define a function to evaluate the performance of a given policy

In [2]:
function evaluate(m::MDP, policy;
                  discount=1.0)
    n = size(m.c, 1)
    c = zeros(n)
    P = zeros(n,n)
        
    for x=1:n
        u = g[x]
        c[x] = m.c[x,u]
        P[x,:] = m.P[u][x,:]
    end
    
    return (eye(n)-discount*P)\c
end

evaluate (generic function with 1 method)

Now we specify the parameters of the model

In [3]:
const rate        = [0.2 0.5 0.8]
const arrivalRate = 0.6

const serviceCost = [0 2 6]
const holdingCost = 2

const R = 1
const M = 8
const A = length(rate)

3

Finally, we construct the cost and probability matrix for the model (This is the same as in Assignment 3)

In [4]:
P = [ spzeros(Float64, M+1, M+1) for u = 1:A]
c = zeros(Float64, M+1, A)

# Initialize cost matrix
c[1,:] = 0 

for x = 1:(M+1)
    for u = 1:A
        if x == 1 
          c[x,u] = serviceCost[u] 
        else
          c[x,u] = (x-1) * holdingCost + serviceCost[u] - R*rate[u]
        end
    end
end

# Initialize Probability matrix
for x = 2:M
    for u = 1:A
        P[u][x, x-1] = (1 - arrivalRate) * rate[u]
        P[u][x, x]   = (1 - arrivalRate) * (1 - rate[u]) + arrivalRate * rate[u]
        P[u][x, x+1] = arrivalRate * ( 1 - rate[u])
    end
end

for u = 1:A
    P[u][1,1] = (1 - arrivalRate) 
    P[u][1,2] = arrivalRate

    P[u][M+1, M+1] = (1 - rate[u])
    P[u][M+1, M  ] = rate[u]
end

In [5]:
model = MDP(c,P)

MDP(9, 3, update!, [0.0 2.0 6.0; 1.8 3.5 7.2; … ; 13.8 15.5 19.2; 15.8 17.5 21.2], Array{Float64,2}[[0.4 0.6 … 0.0 0.0; 0.08 0.44 … 0.0 0.0; … ; 0.0 0.0 … 0.44 0.48; 0.0 0.0 … 0.2 0.8], [0.4 0.6 … 0.0 0.0; 0.2 0.5 … 0.0 0.0; … ; 0.0 0.0 … 0.5 0.3; 0.0 0.0 … 0.5 0.5], [0.4 0.6 … 0.0 0.0; 0.32 0.56 … 0.0 0.0; … ; 0.0 0.0 … 0.56 0.12; 0.0 0.0 … 0.8 0.2]])

Now we write the policy specified in the question. We add `1` because the array indices start from `1` rather than `0`.

In [6]:
g = zeros(Int, M+1)
g[(0:2)+1] = 0
g[(3:5)+1] = 2
g[(6:M)+1] = 1
g = g + 1

9-element Array{Int64,1}:
 1
 1
 1
 3
 3
 3
 2
 2
 2

In [7]:
v = evaluate(model, g; discount=0.9)

9-element Array{Float64,1}:
  56.9486
  67.4947
  80.7094
  92.7984
 107.256 
 122.899 
 137.669 
 148.504 
 153.321 

In [8]:
using Plots
gr()

Plots.GRBackend()

In [9]:
scatter(v, legend = :none)

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 2.5 
 
 
 5.0 
 
 
 7.5 
 
 
 100 
 
 
 150